In [ ]:
!pip install streamlit

In [ ]:
!pip install streamlit_extras

In [ ]:
import streamlit as st

In [ ]:
import streamlit as st
import cv2
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
from streamlit_extras.switch_page_button import switch_page

#################### ALL FUNCTIONS ####################

def plot_image(img):
    fig = plt.figure(figsize=(10,10))
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    return fig

def plot_histogram(img):
    fig = plt.figure(figsize=(10,10))
    plt.hist(img.ravel(), 256, [0,256])
    plt.title("Histogram")
    plt.xlabel("Intensity")
    plt.ylabel("Frequency")
    return fig

def FDF_lowpass(img, D0, n):
    # Converting the image to grayscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Performing the Fourier Transform
    M, N = img.shape # image size
    H = np.zeros((M, N), dtype=np.float32) # filter

    # create the filter
    for u in range(0, M):
        for v in range(0, N):
            D = np.sqrt((u - M/2)**2 + (v - N/2)**2)
            H[u, v] = 1 / (1 + (D/D0)**(2*n))

    # apply the filter
    img_fft = np.fft.fft2(img)
    img_fft_shift = np.fft.fftshift(img_fft)
    img_fft_shift_filt = img_fft_shift * H
    img_fft_filt = np.fft.ifftshift(img_fft_shift_filt)
    img_filt = np.fft.ifft2(img_fft_filt)
    img_filt = np.abs(img_filt)

    # Converting the image back to RGB
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img_filt = cv2.cvtColor(img_filt.astype('float32'), cv2.COLOR_GRAY2RGB)

    return img_filt

def FDF_highpass(img, D0, n):
    # Converting the image to grayscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Performing the Fourier Transform
    M, N = img.shape # image size
    H = np.zeros((M, N), dtype=np.float32) # filter

    # create the filter
    for u in range(0, M):
        for v in range(0, N):
            D = np.sqrt((u - M/2)**2 + (v - N/2)**2)
            H[u, v] = 1 / (1 + (D0/D)**(2*n))

    # apply the filter
    img_fft = np.fft.fft2(img)
    img_fft_shift = np.fft.fftshift(img_fft)
    img_fft_shift_filt = img_fft_shift * H
    img_fft_filt = np.fft.ifftshift(img_fft_shift_filt)
    img_filt = np.fft.ifft2(img_fft_filt)
    img_filt = np.abs(img_filt)

    # Converting the image back to RGB
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img_filt = cv2.cvtColor(img_filt.astype('float32'), cv2.COLOR_GRAY2RGB)

    return img_filt

def FDF_blur(img, kernel_size):
    blur_img = cv2.blur(img, (kernel_size, kernel_size))
    return blur_img

def FDF_sharpen(img, kernel_size):
    kernel_size = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    sharpen_img = cv2.filter2D(img, -1, kernel_size)
    return sharpen_img

def FDF_gaussian_noise(img, mean, stddev):
    #mean = img.mean()
    #stddev = img.std()
    noise = np.random.normal(mean, stddev, img.shape)
    noise_img = img + noise
    return noise_img

def FDF_salt_pepper_noise(img, val):
    sp_noise = np.random.rand(*img.shape) < val
    img_SP = img.copy()
    img_SP[sp_noise] = 255
    return img_SP

def ED_sobel(img, kernel_size):
    sobelx = cv2.Sobel(np.float32(img), cv2.CV_64F, 1, 0, ksize=kernel_size)
    sobely = cv2.Sobel(np.float32(img), cv2.CV_64F, 0, 1, ksize=kernel_size)
    sobelxy = np.sqrt(sobelx**2 + sobely**2)
    return sobelxy

def ED_prewitt(img):
    prewittx = cv2.filter2D(img, -1, np.array([[-1,0,1], [-1,0,1], [-1,0,1]]))
    prewitty = cv2.filter2D(img, -1, np.array([[-1,-1,-1], [0,0,0], [1,1,1]]))
    prewittxy = np.sqrt(prewittx**2 + prewitty**2)
    return prewittxy

def ED_roberts(img):
    robertsx = cv2.filter2D(img, -1, np.array([[0, 0, 0], [0, 1, 0], [0, 0, -1]]))
    robertsy = cv2.filter2D(img, -1, np.array([[0, 0, 0], [0, 0, 1], [0, -1, 0]]))
    robertsxy = np.sqrt(robertsx**2 + robertsy**2)
    return robertsxy

def ED_canny(img, upper_threshold, lower_threshold):
    canny_img = cv2.Canny(img, upper_threshold, lower_threshold)
    return canny_img

def PSNR(img1, img2):
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    psnr = 20 * np.log10(PIXEL_MAX / np.sqrt(mse))
    return psnr

def IS_binary_thresholding(img, threshold):
    ret, thresh_img = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
    return thresh_img

def IS_inverse_binary_thresholding(img, threshold):
    ret, thresh_img = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY_INV)
    return thresh_img

def IS_truncated_thresholding(img, threshold):
    ret, thresh_img = cv2.threshold(img, threshold, 255, cv2.THRESH_TRUNC)
    return thresh_img

def IS_to_zero_thresholding(img, threshold):
    ret, thresh_img = cv2.threshold(img, threshold, 255, cv2.THRESH_TOZERO)
    return thresh_img

def IS_otsu_thresholding(img):
    ret, thresh_img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh_img

def IS_gaussian_thresholding(img):
    ret, thresh_img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh_img

def IS_mean_adaptive_thresholding(img, block_size, constant):
    thresh_img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, constant)
    return thresh_img

def IS_gaussian_adaptive_thresholding(img, block_size, constant):
    thresh_img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, block_size, constant)
    return thresh_img

def RBM_region_growing(img, seed, threshold):
    neighbours = [(0,1), (1,0), (0,-1), (-1,0)]
    region_threshold = threshold
    height, width = img.shape
    visited = np.zeros_like(img)
    segmented_img = np.zeros_like(img)
    stack = []
    stack.append(seed)
    seed_intensity = img[seed]
    while len(stack) > 0:
        current_pixel = stack.pop()
        x, y = current_pixel
        if 0 <= x < height and 0 <= y < width and visited[x,y] == 0:
            intensity_diff = abs(int(img[x,y]) - int(seed_intensity))

            if intensity_diff <= region_threshold:
                segmented_img[x,y] = 255
                visited[x,y] = 1
                for neighbour in neighbours:
                    neighbour_x = x + neighbour[0]
                    neighbour_y = y + neighbour[1]
                    stack.append((neighbour_x, neighbour_y))

    return segmented_img

def RBM_region_splitting(img, threshold):
    height, width = img.shape
    if height <= 1 or width <= 1:
        return img

    mean_value = np.mean(img)
    if mean_value <= threshold:
        return np.zeros_like(img)
    else:
        half_height = height // 2
        half_width = width // 2
        regions = []
        regions.append(img[:half_height, :half_width])
        regions.append(img[:half_height, half_width:])
        regions.append(img[half_height:, :half_width])
        regions.append(img[half_height:, half_width:])
        segmented_regions = [RBM_region_splitting(region, threshold) for region in regions]
        # Ensure all regions have the same dimensions before concatenating
        max_height = max(region.shape[0] for region in segmented_regions)
        max_width = max(region.shape[1] for region in segmented_regions)
        # Resize regions to match the maximum dimensions
        resized_regions = [np.pad(region, ((0, max_height - region.shape[0]), (0, max_width - region.shape[1])), 'constant') for region in segmented_regions]
        segmented_image = np.vstack([np.hstack([resized_regions[0], resized_regions[1]]),
                                     np.hstack([resized_regions[2], resized_regions[3]])])
        return segmented_image


#################### STREAMLIT APP ####################

# Defining the main function
def main():
    # Setting the title of the app

    st.set_page_config(
    page_title="Advanced Image Processor",
    page_icon="🖼️",
    layout="centered",
    initial_sidebar_state="expanded",
    )

    st.title("Advanced Image Processor")

    st.markdown("---")

    #################### SIDEBAR ####################
    st.sidebar.title("💡About")
    st.sidebar.subheader("This is a web app to perform text to image generation and learn about image processing techniques on a given input image.")
    if st.sidebar.button("ℹ️ More Info"):
        switch_page("About and Techniques")

    st.sidebar.title("👨🏽‍💻Developer")
    st.sidebar.info(
        "This app is created by **Mansi Patil**\n"
    )
    ########################################

    uploaded_file = st.file_uploader("Choose an image", type=['jpg', 'png', 'jpeg'])

    if uploaded_file is not None:
        options = ["None", "Fourier Domain Filtering", "Edge Detection", "Image Segmentation", "Region-Based Methods"]
        choice = st.selectbox("Select the method you want to apply", options)

        # Checking if the user has selected a technique
        if choice == "None":
            st.warning("Please select a technique")

        elif choice == "Fourier Domain Filtering":
            technique = st.selectbox("Select the type of filter", ("Low Pass Filter", "High Pass Filter", "Blur Image", "Sharpen Image", "Gaussian Noise", "Salt and Pepper Noise"))

            if technique == "Low Pass Filter":
                D0 = st.slider("Select the cutoff frequency", 0, 100, 30)
                n = st.slider("Select the order of the filter", 1, 10, 2)
                img = np.array(Image.open(uploaded_file))
                img_filt = FDF_lowpass(img, D0, n)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "High Pass Filter":
                D0 = st.slider("Select the cutoff frequency", 0, 100, 30)
                n = st.slider("Select the order of the filter", 1, 10, 2)
                img = np.array(Image.open(uploaded_file))
                img_filt = FDF_highpass(img, D0, n)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "Blur Image":
                kernel_size = st.slider("Select the kernel size", 1, 100, 5)
                img = np.array(Image.open(uploaded_file))
                img_filt = FDF_blur(img, kernel_size)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "Sharpen Image":
                kernel_size = st.slider("Select the kernel size", 1, 100, 5)
                img = np.array(Image.open(uploaded_file))
                img_filt = FDF_sharpen(img, kernel_size)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "Gaussian Noise":
                mean = st.slider("Select the mean", 0, 100, 0)
                stddev = st.slider("Select the standard deviation", 0, 100, 10)
                img = np.array(Image.open(uploaded_file))
                img_filt = FDF_gaussian_noise(img, mean, stddev)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "Salt and Pepper Noise":
                val = st.slider("Select the noise density", 0.0, 1.0, 0.5)
                img = np.array(Image.open(uploaded_file))
                img_filt = FDF_salt_pepper_noise(img, val)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

        elif choice == "Edge Detection":
            technique = st.selectbox("Select the type of filter", ("Sobel Filter", "Prewitt Filter", "Roberts Filter", "Canny Edge Detection"))

            if technique == "Sobel Filter":
                kernel_size = st.slider("Select the kernel size", 1, 31, 5, step=2)
                img = np.array(Image.open(uploaded_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_filt = ED_sobel(img, kernel_size)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "Prewitt Filter":
                img = np.array(Image.open(uploaded_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_filt = ED_prewitt(img)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "Roberts Filter":
                img = np.array(Image.open(uploaded_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_filt = ED_roberts(img)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "Canny Edge Detection":
                upper_threshold = st.slider("Select the upper threshold", 0, 255, 100)
                lower_threshold = st.slider("Select the lower threshold", 0, 255, 50)
                img = np.array(Image.open(uploaded_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_filt = ED_canny(img, upper_threshold, lower_threshold)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

        elif choice == "Image Segmentation":
            technique = st.selectbox("Select the type of filter", ("Binary Thresholding", "Inverse Binary Thresholding", "Truncated Thresholding", "To Zero Thresholding", "Otsu Thresholding", "Gaussian Thresholding", "Mean Adaptive Thresholding", "Gaussian Adaptive Thresholding"))

            if technique == "Binary Thresholding":
                threshold = st.slider("Select the threshold", 0, 255, 100)
                img = np.array(Image.open(uploaded_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_filt = IS_binary_thresholding(img, threshold)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "Inverse Binary Thresholding":
                threshold = st.slider("Select the threshold", 0, 255, 100)
                img = np.array(Image.open(uploaded_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_filt = IS_inverse_binary_thresholding(img, threshold)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "Truncated Thresholding":
                threshold = st.slider("Select the threshold", 0, 255, 100)
                img = np.array(Image.open(uploaded_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_filt = IS_truncated_thresholding(img, threshold)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "To Zero Thresholding":
                threshold = st.slider("Select the threshold", 0, 255, 100)
                img = np.array(Image.open(uploaded_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_filt = IS_to_zero_thresholding(img, threshold)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "Otsu Thresholding":
                img = np.array(Image.open(uploaded_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_filt = IS_otsu_thresholding(img)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "Gaussian Thresholding":
                img = np.array(Image.open(uploaded_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_filt = IS_gaussian_thresholding(img)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "Mean Adaptive Thresholding":
                block_size = st.slider("Select the block size", 3, 255, 5, step=2)
                constant = st.slider("Select the constant", 0, 255, 5)
                img = np.array(Image.open(uploaded_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_filt = IS_mean_adaptive_thresholding(img, block_size, constant)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "Gaussian Adaptive Thresholding":
                block_size = st.slider("Select the block size", 3, 255, 5, step=2)
                constant = st.slider("Select the constant", 0, 255, 5)
                img = np.array(Image.open(uploaded_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_filt = IS_gaussian_adaptive_thresholding(img, block_size, constant)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

        elif choice == "Region-Based Methods":
            technique = st.selectbox("Select the type of filter", ("Region Growing", "Region Splitting"))

            if technique == "Region Growing":
                threshold = st.slider("Select the threshold", 0, 255, 100)
                img = np.array(Image.open(uploaded_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                height, width = img.shape
                seed = (height//2, width//2)
                img_filt = RBM_region_growing(img, seed, threshold)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))

            elif technique == "Region Splitting":
                threshold = st.slider("Select the threshold", 0, 255, 100)
                img = np.array(Image.open(uploaded_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_filt = RBM_region_splitting(img, threshold)
                st.pyplot(plot_image(img_filt))
                try:
                    st.write("PSNR: ", PSNR(img, img_filt))
                except ValueError as ve:
                    st.warning(f"Could not calculate PSNR Value for this method")
                if st.checkbox("Show Histogram"):
                    st.pyplot(plot_histogram(img_filt))


if __name__ == "__main__":
    main()


In [ ]:
! wget -q -O - ipv4.icanhazip.com

34.106.94.161


In [ ]:
! streamlit run Advanced_Image_Processor.py & npx localtunnel --port 8501

[..................] / rollbackFailedOptional: verb npm-session 420a3ffcb6b3e44


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.94.161:8501

npx: installed 22 in 3.294s
your url is: https://fluffy-nails-grow.loca.lt
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [ ]:
!pip install numpy matplotlib opencv-python-headless

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# List of image file names and titles
image_files = ['img1.png', 'img2.png', 'img3.png']

image_titles = ['Chichen Itza', 'Christ The Redeemer', 'Colosseum', 'Great Wall Of China',
                'Machu Picchu', 'Petra', 'Taj Mahal', 'Pyramids']

Displaying images and their Histogram


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Define image files and titles
image_files = ['img1.png', 'img2.png', 'img3.png']
image_titles = ['Image 1', 'Image 2', 'Image 3']

# Create a figure with subplots
fig, axs = plt.subplots(len(image_files), 2, figsize=(10, 10))

# Iterate through each image and title
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    # Read the image
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)

    # Check if img is None (i.e., imread failed)
    if img is None:
        print(f"Failed to read image: {image_file}")
        continue

    img = np.array(img, dtype=np.uint8)  # Convert to np.uint8
    # Plot image on odd column
    axs[i, 0].imshow(img, cmap='gray')
    axs[i, 0].set_title(title)

    # Plot histogram on even column
    axs[i, 1].hist(img.ravel(), 256, [0, 256])
    axs[i, 1].set_title('Histogram of ' + title)


# Show the plot
plt.show()


Butterworth filters


In [ ]:
M, N = img.shape # image size
H = np.zeros((M, N), dtype=np.float32) # filter
D0 = 30 # cutoff frequency
n = 2 # order of the filter

# create the filter
for u in range(0, M):
    for v in range(0, N):
        D = np.sqrt((u - M/2)**2 + (v - N/2)**2)
        H[u, v] = 1 / (1 + (D/D0)**(2*n))

# display the filter
plt.imshow(H, cmap='gray')
plt.title('Butterworth Low Pass Filter')
plt.show()

In [ ]:
# Butterworth High Pass Filter
HPF = 1 - H # high pass filter

# display the filter
plt.imshow(HPF, cmap='gray')
plt.title('Butterworth High Pass Filter')
plt.show()


Fourier Transform


In [ ]:
# Apply Fourier Transform to the image
def fourier_transform(img, H=H, HPF=HPF):
    F = np.fft.fft2(img)
    F_shifted = np.fft.fftshift(F)

    LPF = H * F_shifted # low pass filter
    HPF = HPF * F_shifted # high pass filter

    # inverse Fourier Transform
    LPF_ishifted = np.fft.ifftshift(LPF)
    LPF_image = np.fft.ifft2(LPF_ishifted)
    LPF_image = np.abs(LPF_image)

    HPF_ishifted = np.fft.ifftshift(HPF)
    HPF_image = np.fft.ifft2(HPF_ishifted)
    HPF_image = np.abs(HPF_image)

    # display the images
    fig, axs = plt.subplots(1, 3, figsize=(20, 10))
    axs[0].imshow(img, cmap='gray')
    axs[0].set_title('Original Image')
    axs[1].imshow(LPF_image, cmap='gray')
    axs[1].set_title('Low Pass Filtered Image')
    axs[2].imshow(HPF_image, cmap='gray')
    axs[2].set_title('High Pass Filtered Image')
    plt.show()

# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    # Read the image in grayscale
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    fourier_transform(img)

Blur and sharpen Image

In [ ]:
# Blur images
def blur(img):
    blur_img = cv2.blur(img, (15, 15))
    gauss_blur = cv2.GaussianBlur(img, (15, 15), 0)
    median_blur = cv2.medianBlur(img, 15)

    # display the images
    fig, axs = plt.subplots(2, 2, figsize=(20, 10))
    axs[0, 0].imshow(img, cmap='gray')
    axs[0, 0].set_title('Original Image')
    axs[0, 1].imshow(blur_img, cmap='gray')
    axs[0, 1].set_title('Blur Image')
    axs[1, 0].imshow(gauss_blur, cmap='gray')
    axs[1, 0].set_title('Gaussian Blur Image')
    axs[1, 1].imshow(median_blur, cmap='gray')
    axs[1, 1].set_title('Median Blur Image')
    plt.tight_layout()
    plt.show()

# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    # Read the image in grayscale
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    blur(img)

In [ ]:
# Sharpen images
def sharpen(img):
    # create the sharpening filter
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])

    # applying the sharpening filter
    sharpen_img = cv2.filter2D(img, -1, kernel)

    # display the images
    fig, axs = plt.subplots(1, 2, figsize=(20, 10))
    axs[0].imshow(img, cmap='gray')
    axs[0].set_title('Original Image')
    axs[1].imshow(sharpen_img, cmap='gray')
    axs[1].set_title('Sharpened Image')
    plt.show()

# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    # Read the image in grayscale
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    sharpen(img)

Gussian Noise filters


In [ ]:
# gaussian noise
def gauss_noise(img):
    mean = img.mean()
    stddev = img.std()
    gaussian_noise = np.zeros(img.shape, np.uint8)
    cv2.randn(gaussian_noise, mean, stddev)
    img_gauss = cv2.add(img, gaussian_noise)

    #low pass filter
    lpf_filter3x3 = cv2.blur(img_gauss, (3, 3))
    lpf_filter5x5 = cv2.blur(img_gauss, (5, 5))
    lpf_filter7x7 = cv2.blur(img_gauss, (7, 7))
    lpf_filter9x9 = cv2.blur(img_gauss, (9, 9))

    # display the images
    fig, axs = plt.subplots(2, 3, figsize=(20, 10))
    axs[0, 0].imshow(img, cmap='gray')
    axs[0, 0].set_title('Original Image')
    axs[0, 1].imshow(img_gauss, cmap='gray')
    axs[0, 1].set_title('Gaussian Noise')
    axs[0, 2].imshow(lpf_filter3x3, cmap='gray')
    axs[0, 2].set_title('3x3 Low Pass Filter')
    axs[1, 0].imshow(lpf_filter5x5, cmap='gray')
    axs[1, 0].set_title('5x5 Low Pass Filter')
    axs[1, 1].imshow(lpf_filter7x7, cmap='gray')
    axs[1, 1].set_title('7x7 Low Pass Filter')
    axs[1, 2].imshow(lpf_filter9x9, cmap='gray')
    axs[1, 2].set_title('9x9 Low Pass Filter')
    plt.show()


In [ ]:
# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    # Read the image in grayscale
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    gauss_noise(img)

Salt and Paper noise


In [ ]:
# salt and pepper noise
def salt_pepper(img):
    # salt and pepper noise
    sp_noise = np.zeros(img.shape, np.uint8)
    cv2.randu(sp_noise, 0, 255)
    sp_noise = sp_noise < 255 * 0.05
    img_sp = img.copy()
    img_sp[sp_noise] = 255

    #low pass filter
    lpf_filter3x3 = cv2.blur(img_sp, (3, 3))
    lpf_filter5x5 = cv2.blur(img_sp, (5, 5))
    lpf_filter7x7 = cv2.blur(img_sp, (7, 7))
    lpf_filter9x9 = cv2.blur(img_sp, (9, 9))

    # display the images
    fig, axs = plt.subplots(2, 3, figsize=(20, 10))
    axs[0, 0].imshow(img, cmap='gray')
    axs[0, 0].set_title('Original Image')
    axs[0, 1].imshow(img_sp, cmap='gray')
    axs[0, 1].set_title('Salt and Pepper Noise')
    axs[0, 2].imshow(lpf_filter3x3, cmap='gray')
    axs[0, 2].set_title('3x3 Low Pass Filter')
    axs[1, 0].imshow(lpf_filter5x5, cmap='gray')
    axs[1, 0].set_title('5x5 Low Pass Filter')
    axs[1, 1].imshow(lpf_filter7x7, cmap='gray')
    axs[1, 1].set_title('7x7 Low Pass Filter')
    axs[1, 2].imshow(lpf_filter9x9, cmap='gray')
    axs[1, 2].set_title('9x9 Low Pass Filter')
    plt.show()

In [ ]:
# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    # Read the image in grayscale
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    salt_pepper(img)

## **EDGE DETECTION (NANDINI)**

Sobel Edge Detection


In [ ]:
# Applying Sobel edge detection
def sobel(img):
    sobelx = cv2.Sobel(np.float64(img), cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(np.float64(img), cv2.CV_64F, 0, 1, ksize=5)
    sobelxy = np.sqrt(sobelx**2 + sobely**2)

    # display the images
    fig, axs = plt.subplots(1, 4, figsize=(20, 10))
    axs[0].imshow(img, cmap='gray')
    axs[0].set_title('Original Image')
    axs[1].imshow(sobelx, cmap='gray')
    axs[1].set_title('Sobel X')
    axs[2].imshow(sobely, cmap='gray')
    axs[2].set_title('Sobel Y')
    axs[3].imshow(sobelxy, cmap='gray')
    axs[3].set_title('Sobel XY')
    plt.show()

# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    # Read the image in grayscale
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    sobel(img)

In [ ]:
# Applying Prewitt edge detection
def prewitt(img):
    prewittx = cv2.filter2D(img, -1, np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]))
    prewitty = cv2.filter2D(img, -1, np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]]))
    prewittxy = np.sqrt(prewittx**2 + prewitty**2)

    # display the images
    fig, axs = plt.subplots(1, 4, figsize=(20, 10))
    axs[0].imshow(img, cmap='gray')
    axs[0].set_title('Original Image')
    axs[1].imshow(prewittx, cmap='gray')
    axs[1].set_title('Prewitt X')
    axs[2].imshow(prewitty, cmap='gray')
    axs[2].set_title('Prewitt Y')
    axs[3].imshow(prewittxy, cmap='gray')
    axs[3].set_title('Prewitt XY')
    plt.show()

# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    # Read the image in grayscale
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    prewitt(img)

Robert Edge Detection


In [ ]:
# Applying Roberts edge detection
def roberts(img):
    robertsx = cv2.filter2D(img, -1, np.array([[0, 0, 0], [0, 1, 0], [0, 0, -1]]))
    robertsy = cv2.filter2D(img, -1, np.array([[0, 0, 0], [0, 0, 1], [0, -1, 0]]))
    robertsxy = np.sqrt(robertsx**2 + robertsy**2)

    # display the images
    fig, axs = plt.subplots(1, 4, figsize=(20, 10))
    axs[0].imshow(img, cmap='gray')
    axs[0].set_title('Original Image')
    axs[1].imshow(robertsx, cmap='gray')
    axs[1].set_title('Roberts X')
    axs[2].imshow(robertsy, cmap='gray')
    axs[2].set_title('Roberts Y')
    axs[3].imshow(robertsxy, cmap='gray')
    axs[3].set_title('Roberts XY')
    plt.show()

# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    # Read the image in grayscale
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    roberts(img)

Canny Edge Detection

In [ ]:
### Applying canny edge detection
def canny(img):
    lower_threshold = 100
    upper_threshold = 200
    canny = cv2.Canny(img, lower_threshold, upper_threshold)

    # display the images
    fig, axs = plt.subplots(1, 2, figsize=(20, 10))
    axs[0].imshow(img, cmap='gray')
    axs[0].set_title('Original Image')
    axs[1].imshow(canny, cmap='gray')
    axs[1].set_title('Canny Edge Detection')
    plt.show()

# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    # Read the image in grayscale
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    canny(img)

## **IMAGE SEGMENTATION** (SAURABH)

In [ ]:
#PEAK SIGNAL TO NOISE RATIO Image segmentation

In [ ]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

binary threshold


In [ ]:
# Apply Binary Thresholding
def bin_thresholding(img):
    thresh_val = 127
    ret, thresh1 = cv2.threshold(img, thresh_val, 255, cv2.THRESH_BINARY)
    ret, thresh2 = cv2.threshold(img, thresh_val, 255, cv2.THRESH_BINARY_INV)

    psnr_val1 = psnr(img, thresh1)
    psnr_val2 = psnr(img, thresh2)
    print("----------------------------------------------")
    print('PSNR value for Binary Thresholding: ', psnr_val1)
    print('PSNR value for Binary Thresholding Inverse: ', psnr_val2)
    print("----------------------------------------------")
   # display the images and their histograms
    fig, axs = plt.subplots(2, 3, figsize=(20, 10))
    axs[0, 0].imshow(img, cmap='gray')
    axs[0, 0].set_title('Original Image')
    axs[0, 1].imshow(thresh1, cmap='gray')
    axs[0, 1].set_title('Binary Thresholding')
    axs[0, 2].imshow(thresh2, cmap='gray')
    axs[0, 2].set_title('Binary Thresholding Inverse')
    axs[1, 0].hist(img.ravel(), 256, [0, 256])
    axs[1, 1].hist(thresh1.ravel(), 256, [0, 256])
    axs[1, 2].hist(thresh2.ravel(), 256, [0, 256])
    plt.tight_layout()
    plt.show()

# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    bin_thresholding(img)

truncated thresholding

In [ ]:
# Applying Trunc Thresholding
def trunc(img):
    thresh_val = img.mean()
    ret, thresh1 = cv2.threshold(img, thresh_val, 255, cv2.THRESH_TRUNC)

    psnr_val1 = psnr(img, thresh1)

    print("----------------------------------------------")
    print('PSNR value for Truncated Thresholding: ', psnr_val1)
    print("----------------------------------------------")

    # display the images and their histograms
    fig, axs = plt.subplots(2, 2, figsize=(20, 10))
    axs[0, 0].imshow(img, cmap='gray')
    axs[0, 0].set_title('Original Image')
    axs[0, 1].imshow(thresh1, cmap='gray')
    axs[0, 1].set_title('Trunc Thresholding')
    axs[1, 0].hist(img.ravel(), 256, [0, 256])
    axs[1, 1].hist(thresh1.ravel(), 256, [0, 256])
    plt.tight_layout()
    plt.show()

# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    trunc(img)

In [ ]:
# Applying to zero thresholding
def toZero(img):
    thresh_val = img.mean()
    ret, thresh1 = cv2.threshold(img, thresh_val, 255, cv2.THRESH_TOZERO)

    psnr_val1 = psnr(img, thresh1)
    print("----------------------------------------------")
    print('PSNR value for To Zero Thresholding: ', psnr_val1)
    print("----------------------------------------------")

    # display the images and their histograms
    fig, axs = plt.subplots(2, 2, figsize=(20, 10))
    axs[0, 0].imshow(img, cmap='gray')
    axs[0, 0].set_title('Original Image')
    axs[0, 1].imshow(thresh1, cmap='gray')
    axs[0, 1].set_title('To Zero Thresholding')
    axs[1, 0].hist(img.ravel(), 256, [0, 256])
    axs[1, 1].hist(thresh1.ravel(), 256, [0, 256])
    plt.tight_layout()
    plt.show()

# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    toZero(img)

In [ ]:
# Applying Otsu's Thresholding
def otsu(img):
    ret, thresh1 = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    psnr_val1 = psnr(img, thresh1)
    print("----------------------------------------------")
    print('PSNR value for Otsu Thresholding: ', psnr_val1)
    print("----------------------------------------------")

    # display the images and their histograms
    fig, axs = plt.subplots(2, 2, figsize=(20, 10))
    axs[0, 0].imshow(img, cmap='gray')
    axs[0, 0].set_title('Original Image')
    axs[0, 1].imshow(thresh1, cmap='gray')
    axs[0, 1].set_title('Otsu Thresholding')
    axs[1, 0].hist(img.ravel(), 256, [0, 256])
    axs[1, 1].hist(thresh1.ravel(), 256, [0, 256])
    plt.tight_layout()
    plt.show()

# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    otsu(img)

In [ ]:
# Applying Gaussian Thresholding
def gauss_thresh(img):
    ret, thresh1 = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    ret, thresh2 = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_TRIANGLE)

    psnr_val1 = psnr(img, thresh1)
    psnr_val2 = psnr(img, thresh2)
    print("----------------------------------------------")
    print('PSNR value for Otsu Gaussian Thresholding: ', psnr_val1)
    print('PSNR value for Triangle Gaussian Thresholding: ', psnr_val2)
    print("----------------------------------------------")

    # display the images and their histograms
    fig, axs = plt.subplots(2, 3, figsize=(20, 10))
    axs[0, 0].imshow(img, cmap='gray')
    axs[0, 0].set_title('Original Image')
    axs[0, 1].imshow(thresh1, cmap='gray')
    axs[0, 1].set_title('Otsu Gaussian Thresholding')
    axs[0, 2].imshow(thresh2, cmap='gray')
    axs[0, 2].set_title('Triangle Gaussian Thresholding')
    axs[1, 0].hist(img.ravel(), 256, [0, 256])
    axs[1, 1].hist(thresh1.ravel(), 256, [0, 256])
    axs[1, 2].hist(thresh2.ravel(), 256, [0, 256])
    plt.tight_layout()
    plt.show()

# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    gauss_thresh(img)

In [ ]:
# Applying Adaptive Thresholding
def adaptive_thresh(img):
    thresh1 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    thresh2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    psnr_val1 = psnr(img, thresh1)
    psnr_val2 = psnr(img, thresh2)
    print("----------------------------------------------")
    print('PSNR value for Mean Adaptive Thresholding: ', psnr_val1)
    print('PSNR value for Gaussian Adaptive Thresholding: ', psnr_val2)
    print("----------------------------------------------")

    # display the images and their histograms
    fig, axs = plt.subplots(2, 3, figsize=(20, 10))
    axs[0, 0].imshow(img, cmap='gray')
    axs[0, 0].set_title('Original Image')
    axs[0, 1].imshow(thresh1, cmap='gray')
    axs[0, 1].set_title('Mean Adaptive Thresholding')
    axs[0, 2].imshow(thresh2, cmap='gray')
    axs[0, 2].set_title('Gaussian Adaptive Thresholding')
    axs[1, 0].hist(img.ravel(), 256, [0, 256])
    axs[1, 1].hist(thresh1.ravel(), 256, [0, 256])
    axs[1, 2].hist(thresh2.ravel(), 256, [0, 256])
    plt.tight_layout()
    plt.show()

# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    adaptive_thresh(img)

In [ ]:
def region_growing(image, seed, threshold):
    neighbours = [(1,0),(-1,0),(0,1),(0,-1)]
    region_threshold = threshold

    height, width = image.shape

    visited = np.zeros_like(image)

    segmented = np.zeros_like(image)

    stack = []
    stack.append(seed)

    seed_intensity = image[seed]

    while len(stack) > 0:
        current_pixel = stack.pop()
        x, y = current_pixel

        if 0 <= x < height and 0 <= y < width and visited[x,y] == 0:
            intensity_diff = abs(int(image[x,y]) - int(seed_intensity))

            if intensity_diff < region_threshold:
                segmented[x,y] = 255
                visited[x,y] = 1

                for neighbour in neighbours:
                    neighbour_x = x + neighbour[0]
                    neighbour_y = y + neighbour[1]
                    stack.append((neighbour_x, neighbour_y))

    return segmented

def region_growing_segmentation(img, seed=(200, 600), threshold=75):
    segmented = region_growing(img, seed, threshold)

    # display the images and their histograms
    fig, axs = plt.subplots(1, 2, figsize=(20, 10))
    axs[0].imshow(img, cmap='gray')
    axs[0].set_title('Original Image')
    axs[1].imshow(segmented, cmap='gray')
    axs[1].set_title('Region Growing Segmentation')
    plt.tight_layout()
    plt.show()

In [ ]:

print("REGION GROWING 1")
# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    region_growing_segmentation(img, seed=(200, 600), threshold=75)
print("---------------------------------------------------------------------")

In [ ]:
print("REGION GROWING 2")
# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    region_growing_segmentation(img, seed=(600, 600), threshold=70)
print("---------------------------------------------------------------------")

Region Splitting


In [ ]:
# Applying region split technique

def region_split(image, threshold):
    height, width = image.shape
    if height <= 1 or width <= 1:
        return image

    mean_value = np.mean(image)
    if mean_value <= threshold:
        return np.zeros_like(image)
    else:
        half_height = height // 2
        half_width = width // 2
        regions = []
        regions.append(image[:half_height, :half_width])
        regions.append(image[:half_height, half_width:])
        regions.append(image[half_height:, :half_width])
        regions.append(image[half_height:, half_width:])
        segmented_regions = [region_split(region, threshold) for region in regions]
        # Ensure all regions have the same dimensions before concatenating
        max_height = max(region.shape[0] for region in segmented_regions)
        max_width = max(region.shape[1] for region in segmented_regions)
        # Resize regions to match the maximum dimensions
        resized_regions = [np.pad(region, ((0, max_height - region.shape[0]), (0, max_width - region.shape[1])), 'constant') for region in segmented_regions]
        segmented_image = np.vstack([np.hstack([resized_regions[0], resized_regions[1]]),
                                     np.hstack([resized_regions[2], resized_regions[3]])])
        return segmented_image

def region_split_segmentation(img, threshold=50):
    segmented = region_split(img, threshold)

    # display the images
    fig, axs = plt.subplots(1, 2, figsize=(20, 10))
    axs[0].imshow(img, cmap='gray')
    axs[0].set_title('Original Image')
    axs[1].imshow(segmented, cmap='gray')
    axs[1].set_title('Region Splitting Segmentation')
    plt.tight_layout()
    plt.show()

print("REGION SPLITTING 1")
# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    region_split_segmentation(img, threshold=50)



In [ ]:
# Applying region split and merging technique

def region_split(image, threshold):
    height, width = image.shape
    if height <= 1 or width <= 1:
        return image

    mean_value = np.mean(image)
    if mean_value <= threshold:
        return np.zeros_like(image)
    else:
        half_height = height // 2
        half_width = width // 2
        regions = []
        regions.append(image[:half_height, :half_width])
        regions.append(image[:half_height, half_width:])
        regions.append(image[half_height:, :half_width])
        regions.append(image[half_height:, half_width:])
        segmented_regions = [region_split(region, threshold) for region in regions]
        # Ensure all regions have the same dimensions before concatenating
        max_height = max(region.shape[0] for region in segmented_regions)
        max_width = max(region.shape[1] for region in segmented_regions)
        # Resize regions to match the maximum dimensions
        resized_regions = [cv2.resize(region, (max_width, max_height)) for region in segmented_regions]
        segmented_image = np.vstack([np.hstack([resized_regions[0], resized_regions[1]]),
                                     np.hstack([resized_regions[2], resized_regions[3]])])
        return segmented_image

def region_split_segmentation(img, threshold=50):
    segmented = region_split(img, threshold)

    # display the images
    fig, axs = plt.subplots(1, 2, figsize=(20, 10))
    axs[0].imshow(img, cmap='gray')
    axs[0].set_title('Original Image')
    axs[1].imshow(segmented, cmap='gray')
    axs[1].set_title('Region Splitting and Merging Segmentation')
    plt.tight_layout()
    plt.show()


print("REGION SPLITTING 1")
# Applying the filter to the images
for i, (image_file, title) in enumerate(zip(image_files, image_titles)):
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    print(title)
    region_split_segmentation(img, threshold=100)
